## Predict HFpEF two-year re-admission

use trained model to predict the re-admission.

Make sure you have run/read step1 code.

---

### Docker environment
Please use `docker/docker_torch`, make sure you install ```torch_geometric``` package


In [1]:
import sys
sys.path.append("/workspace/Documents") 
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
import matplotlib.pyplot as plt
import ast
from itertools import combinations
from itertools import chain

import pandas as pd
import os
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
import HFpEF_CMR_GraphStrain.functions_collection as ff
import HFpEF_CMR_GraphStrain.Data_processing as dp
import HFpEF_CMR_GraphStrain.helpers.load_data as load_data
import HFpEF_CMR_GraphStrain.helpers.generator as generator
import HFpEF_CMR_GraphStrain.helpers.train_func as train_func
import HFpEF_CMR_GraphStrain.helpers.STGCN as STGCN
import HFpEF_CMR_GraphStrain.helpers.best_epoch as best_epoch


main_path = '/mnt/camca_NAS/Deepstrain/'

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Section 1: process the regional LV strain

1. our default time frame number = 25. Running this code will interpolate time frames if original one is not 25

2. our strains were saved for each time frame. running this code will process them in a big matrix

In [2]:
# define the patient list 
patient_list = pd.read_excel(os.path.join(main_path, 'example_data/Patient_list/patient_list.xlsx'))
patient_list = patient_list[patient_list['include?'] == 'yes']
print(patient_list.shape)

for i in range(0, patient_list.shape[0]):
    patient_id = ff.XX_to_ID_00XX(patient_list.iloc[i, 0])

    # put into a big matrix
    folders = ff.sort_timeframe(ff.find_all_target_files(['tf_*'],os.path.join(main_path,'example_data/results/strain',patient_id)), 0,'_','')
    Ecc_aha_ori = np.zeros((len(folders),16))
    Err_aha_ori = np.zeros((len(folders),16))

    for i in range(len(folders)):
        strain_file = np.load(os.path.join(folders[i],'strain_info.npy'),allow_pickle=True)
        ecc_aha = np.asarray(strain_file[-2][:-1] )
        err_aha = np.asarray(strain_file[-1][:-1] )
        Ecc_aha_ori[i] = ecc_aha
        Err_aha_ori[i] = err_aha

    Ecc_aha = np.zeros((16, len(folders))); Err_aha = np.zeros((16, len(folders)))
    for i in range(0,16):
        for j in range(len(folders)):
            Ecc_aha[i,j] = Ecc_aha_ori[j,i]
            Err_aha[i,j] = Err_aha_ori[j,i]

    print('original shape:', Ecc_aha.shape, Err_aha.shape)

    # sample 25 time frames
    x_original = np.arange(0,Ecc_aha.shape[1])
    x_new = np.linspace(0, Ecc_aha.shape[1]-1, 25)  # 25 time frames

    Ecc_aha_sample = np.zeros((16,25))
    Err_aha_sample = np.zeros((16,25))

    for i in range(16):
        spl_ecc = make_interp_spline(x_original, Ecc_aha[i,:], k=2)  # k=3表示三次样条
        ecc_new = spl_ecc(x_new)

        spl_err = make_interp_spline(x_original, Err_aha[i,:], k=2)  # k=3表示三次样条
        err_new = spl_err(x_new)

        Ecc_aha_sample[i] = ecc_new
        Err_aha_sample[i] = err_new

    np.save(os.path.join(main_path, 'example_data/results/strain', patient_id, 'Ecc_aha_sample.npy'), Ecc_aha_sample)
    np.save(os.path.join(main_path, 'example_data/results/strain', patient_id, 'Err_aha_sample.npy'), Err_aha_sample)



(1, 5)
original shape: (16, 15) (16, 15)


### Section 2: Train Graph Convolutional Network
we can also add EHR data if needed

In [2]:
# load data
label_sheet = pd.read_excel(os.path.join(main_path, 'example_data/Patient_list/patient_list.xlsx'))

strain_loader = load_data.Load(os.path.join(main_path, 'example_data/results/'))
X_Ecc, X_Err, Y = strain_loader.load_strain(label_sheet)
print('X_Ecc.shape:', X_Ecc.shape, ' X_Err.shape:', X_Err.shape, ' Y.shape:', Y.shape)

# load EHR if needed
ehr_sheet = pd.read_excel(os.path.join(main_path, 'example_data/Patient_list/patient_clinical_data.xlsx'))

ehr_loader = load_data.Load(main_path)
ehr_array, features = ehr_loader.load_ehr(ehr_sheet)
print('ehr_array.shape:', ehr_array.shape, ' features:', features)

shape of Ecc, Err: (16, 25) (12, 25)
X_Ecc.shape: (1, 16, 25)  X_Err.shape: (1, 12, 25)  Y.shape: (1,)
ehr_array.shape: (1, 27)
ehr_array.shape: (1, 27)  features: ['Age', 'Sex', 'BloodPressureSystolic', 'BloodPressureDiastolic', 'HeartRate', 'Weight', 'BMI', 'HGB', 'Glucose', 'Sodium', 'BUN', 'AtrialFibrillation', 'CoronaryVascularDisease', 'MyocardialInfarction', 'Diabetes', 'Hypertension', 'ChronicKidneyDisease', 'Obesity', 'Anemia', 'Cardiomyopathy', 'MetabolicSyndrome', 'Neoplasm', 'Diuretics', 'BetaBlockers', 'ACEI', 'CCB', 'Statin']


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [3]:
# define GCN
# define the edge matrix
edge_index_ecc, edge_index_err = STGCN.edge_index_cal()

torch.Size([2, 64])
torch.Size([2, 36])


In [6]:
# train
# as example code, we train and validate model on the same one case. 
# please use your own data to train and validate the model.

model_name = 'GCN_Ecc_Err' # GCN_Ecc_Err_EHR if you want to add clinical data as input
strain = 'both' # define whether to use Ecc, Err, or both
save_folder = os.path.join(main_path, 'example_data/models/', model_name, 'models')
ff.make_folder([os.path.join(main_path,'example_data/models'), os.path.join(main_path,'example_data/models', model_name), save_folder, os.path.join(main_path,'example_data/models', model_name, 'log')])

train,val = [0], [0]
# define Y_train
Y_train = Y[train]
# count positive and negative samples
num_neg,num_pos= (Y_train == 0).sum().item(),(Y_train == 1).sum().item()
pos_weight_value =  torch.tensor([1.], dtype=torch.float32) #torch.tensor([num_neg / num_pos], dtype=torch.float32)
  
# weight loss
weighted_bce_loss = train_func.WeightedBCELoss(pos_weight_value)

# define data generator
generator_train = generator.Data_generator(X_Ecc, X_Err, Y, train, shuffle = True) if 'EHR' not in model_name else generator.Data_generator(X_Ecc, X_Err, Y, train, shuffle = True, EHR = ehr_array)
generator_val = generator.Data_generator(X_Ecc, X_Err, Y, val, shuffle = False) if 'EHR' not in model_name else generator.Data_generator(X_Ecc, X_Err, Y, val, shuffle = False, EHR = ehr_array)

if 'EHR' not in model_name:
  # define model, parameters are default (no need to change)
  if strain == 'Ecc' or strain == 'Err' :
    model = STGCN.STGCN(strain = strain,GCN_type ='ChebConv', Cheb_K = 2, aha_num = 16, tf_num = 25, temporal_out = 0, hidden_size=128, dropout_rate=0.3, edge_index_ecc= edge_index_ecc , edge_index_err = edge_index_err, get_latent_layer = False)
  elif strain == 'both':
    model = STGCN.DualSTGCN(fusion_method= 'gated', GCN_type = 'ChebConv', Cheb_K = 2, aha_num = 16, tf_num = 25, temporal_out= 0,  hidden_size=128, dropout_rate=0.3, edge_index_ecc = edge_index_ecc, edge_index_err = edge_index_err)
  
if 'EHR' in model_name: # use Ecc, Err and EHR
    model = STGCN.DualSTGCN_w_EHR(fusion_method= 'gated', ehr_dim = ehr_array.shape[-1], ehr_out = 128, GCN_type = 'ChebConv', Cheb_K = 2, aha_num = 16, tf_num = 25, temporal_out= 0,  hidden_size=128, dropout_rate=0.3, edge_index_ecc = edge_index_ecc, edge_index_err = edge_index_err)
      
# train
trainer = train_func.Trainer(model, 
                            weighted_bce_loss, 
                            generator_train, 
                            generator_val, 
                            train_batch_size = 1, 
                            train_num_steps = 2000, 
                            train_lr_decay_every = 1000,
                            save_folder = save_folder, 
                            save_models_every = 10,
                            lr = 0.0001)
trainer.train()

### Section 3: Predict using trained network

In [14]:
# load data
label_sheet = pd.read_excel(os.path.join(main_path, 'example_data/Patient_list/patient_list.xlsx'))

strain_loader = load_data.Load(os.path.join(main_path, 'example_data/results/'))
X_Ecc, X_Err, Y = strain_loader.load_strain(label_sheet)
print('X_Ecc.shape:', X_Ecc.shape, ' X_Err.shape:', X_Err.shape, ' Y.shape:', Y.shape)

# load EHR if needed
ehr_sheet = pd.read_excel(os.path.join(main_path, 'example_data/Patient_list/patient_clinical_data.xlsx'))

ehr_loader = load_data.Load(main_path)
ehr_array, features = ehr_loader.load_ehr(ehr_sheet)
print('ehr_array.shape:', ehr_array.shape, ' features:', features)

shape of Ecc, Err: (16, 25) (12, 25)
X_Ecc.shape: (1, 16, 25)  X_Err.shape: (1, 12, 25)  Y.shape: (1,)
ehr_array.shape: (1, 27)
ehr_array.shape: (1, 27)  features: ['Age', 'Sex', 'BloodPressureSystolic', 'BloodPressureDiastolic', 'HeartRate', 'Weight', 'BMI', 'HGB', 'Glucose', 'Sodium', 'BUN', 'AtrialFibrillation', 'CoronaryVascularDisease', 'MyocardialInfarction', 'Diabetes', 'Hypertension', 'ChronicKidneyDisease', 'Obesity', 'Anemia', 'Cardiomyopathy', 'MetabolicSyndrome', 'Neoplasm', 'Diuretics', 'BetaBlockers', 'ACEI', 'CCB', 'Statin']


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [15]:
# define GCN
# define the edge matrix
edge_index_ecc, edge_index_err = STGCN.edge_index_cal()

torch.Size([2, 64])
torch.Size([2, 36])


In [17]:
# predict 
model_name = 'GCN_Ecc_Err'
strain = 'both'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define file
trained_model_filename = os.path.join(main_path, 'example_data/models', model_name, 'models', 'model-final.pt')

val = [0]
# define generator
generator_val = generator.Data_generator(X_Ecc, X_Err, Y, val, shuffle = False) if 'EHR' not in model_name else generator.Data_generator(X_Ecc, X_Err, Y, val, shuffle = False, EHR = ehr_array)
dl_val= generator.DataLoader(generator_val, batch_size = 1, shuffle = False, pin_memory = True, num_workers = 0)

# define model
if 'EHR' not in model_name:
  # define model, parameters are default (no need to change)
  if strain == 'Ecc' or strain == 'Err' :
    model = STGCN.STGCN(strain = strain,GCN_type ='ChebConv', Cheb_K = 2, aha_num = 16, tf_num = 25, temporal_out = 0, hidden_size=128, dropout_rate=0.3, edge_index_ecc= edge_index_ecc , edge_index_err = edge_index_err, get_latent_layer = False)
  elif strain == 'both':
    model = STGCN.DualSTGCN(fusion_method= 'gated', GCN_type = 'ChebConv', Cheb_K = 2, aha_num = 16, tf_num = 25, temporal_out= 0,  hidden_size=128, dropout_rate=0.3, edge_index_ecc = edge_index_ecc, edge_index_err = edge_index_err)
  
if 'EHR' in model_name: # use Ecc, Err and EHR
    model = STGCN.DualSTGCN_w_EHR(fusion_method= 'gated', ehr_dim = ehr_array.shape[-1], ehr_out = 128, GCN_type = 'ChebConv', Cheb_K = 2, aha_num = 16, tf_num = 25, temporal_out= 0,  hidden_size=128, dropout_rate=0.3, edge_index_ecc = edge_index_ecc, edge_index_err = edge_index_err)
      

trainer = train_func.Trainer(model, None, generator_val, generator_val, train_batch_size = 1, train_num_steps = 1, save_folder = os.path.join(main_path, 'example_data/models'), train_lr_decay_every = 1, save_models_every = 1)
trainer.load_model(trained_model_filename)
model.eval()

for batch in dl_val:
    batch_Ecc, batch_Err, batch_Err_padded, batch_y , batch_ehr = batch
    data_Ecc, data_Err, data_Err_padded, data_y, data_ehr = batch_Ecc.to(device), batch_Err.to(device), batch_Err_padded.to(device), batch_y.to(device), batch_ehr.to(device)
    # predict
    with torch.no_grad():
        if strain == 'Err':
            y_pred_prob = model(data_Err)
        elif strain == 'Ecc':
            y_pred_prob = model(data_Ecc)
        else:
            if 'EHR' not in model_name:
                y_pred_prob  = model(data_Ecc, data_Err)
            else:
                y_pred_prob = model(data_Ecc, data_Err, data_ehr)

        print('y_pred_prob:', y_pred_prob.item())
        print('y true:', data_y.item())

y_pred_prob: 0.929652750492096
y true: 1.0
